In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def chunker(seq, size):
    for pos in range(0, len(seq), size):
        yield seq.iloc[pos:pos + size] 

# Read Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/craft-cans/beers.csv', index_col=0)
df = df.rename(index=int)
df = df.dropna()

In [ ]:
df = df.drop(['brewery_id', 'id'], axis=1)

In [ ]:
chnk = chunker(df.reset_index(), 1000)

In [ ]:
db_name = "beers2021"

In [ ]:
for df_chnk in chnk:
    partial_chunk = df_chnk.to_json(orient='records')
    r = requests.post(url=f'https://mycelia.azure-api.net/clone/table/setup/data/{db_name}',data=partial_chunk)
    print(r.status_code)

In [ ]:
r = requests.post(url=f'https://mycelia.azure-api.net/clone/table/setup/unsupervised/{db_name}')
print(r.status_code)

## Define Similarity Query Parameters

### Query: Beer code (index) to query
### top_k: Number of beer lookalikes to return

In [ ]:
top_k = 10
query = 1439
df.loc[query]

## Run Similarity Search Query

In [ ]:
ans = requests.get(f'https://mycelia.azure-api.net/clone/similar/id/{db_name}?index={query}&top_k={top_k}')

In [ ]:
ans

## Parse and print resuts

In [ ]:
parsed_dict = ans.json()['similarity'][0]['results']
similar_beers = pd.DataFrame.from_dict(parsed_dict, orient='columns').set_index('index')
similar_beers = similar_beers.rename(index=int)

In [ ]:
df_ans = similar_beers.merge(df, right_index=True, left_index=True).sort_values('distance')

In [ ]:
df_ans

In [ ]:
print(df_ans)